En este notebook usaremos llama 3 a través de una API (uso "limitado" gratuito), sin usar el llm localmente en la computadora. Es necesario crear una cuenta aquí:

https://console.groq.com/login

y solicitar una key:

https://console.groq.com/keys

Create a file named .env where you store the value of your key:

GROQ_API_KEY='---'

In [1]:
!pip3 install groq

In [2]:
!pip3 install PyPDF2 langchain==0.3.9 langchain-community==0.3.9 langchain-core==0.3.29 langchain-experimental==0.3.3 huggingface-hub==0.23.2 instructor==1.7.0 InstructorEmbedding==1.0.1 sentence-transformers==2.2.2 torch pdfplumber faiss-cpu


In [3]:
!pip3 install langchain_groq

In [4]:
from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.document_loaders import TextLoader, PyPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_experimental.text_splitter import SemanticChunker
import os
from langchain.embeddings import OllamaEmbeddings
import pdfplumber
from langchain_community.embeddings import HuggingFaceInstructEmbeddings
from InstructorEmbedding import INSTRUCTOR
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_ollama import ChatOllama
from langchain_ollama.llms import OllamaLLM
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain_groq import ChatGroq
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.embeddings import SentenceTransformerEmbeddings



c:\Users\L01356196\Downloads\CADI_2025_LLMs-main\.venv\Lib\site-packages\InstructorEmbedding\instructor.py:7: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import trange


In [5]:
from dotenv import load_dotenv
load_dotenv('/Users/L01356196/Downloads/CADI_2025_LLMs-main/CADI_2025_LLMs-main/.env')
api_key= os.getenv('GROQ_API_KEY')

In [7]:
model_name = 'llama-3.1-70b-versatile'
#model_name = 'llama3-8b-8192'

groq_chat = ChatGroq(
        groq_api_key=api_key,
        model_name=model_name
    )

llm = groq_chat

In [8]:
## Do not mofify
def load_db(embeddings,pdf_path):
    text =''
    with open(pdf_path,'rb') as file:
        pdf_reader = PdfReader(file)
        for page in pdf_reader.pages:
            text += page.extract_text()

    text_splitter=SemanticChunker(
        embeddings, breakpoint_threshold_type="percentile")
    #text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
    docs = text_splitter.split_text(text)
    # define embedding
    vectorstore = FAISS.from_texts(docs, embeddings)
    return vectorstore

In [9]:
#embeddings = HuggingFaceInstructEmbeddings(model_name='hkunlp/instructor-xl') #Problems with Python 12
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

C:\Users\L01356196\AppData\Local\Temp\ipykernel_27964\3581151796.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")


In [10]:
pdf_path = "IoT8815.pdf"

In [11]:
#Do not modify 
import os
if not os.path.exists('faiss_index'):
    #vectorstore=load_db(embeddings,pdf_path)
    vectorstore=load_db(embeddings,'/Users/L01356196/Downloads/CADI_2025_LLMs-main/IoT8815.pdf')
    vectorstore.save_local("faiss_index")
else:
    vectorstore = FAISS.load_local("faiss_index",embeddings=embeddings,allow_dangerous_deserialization=True)


In [12]:
retriever = vectorstore.as_retriever()

In [13]:
template = """Eres un asistente para contestar preguntas del ususario sobre el contenido del file subido, que trata de un artículo sobre la implementación de un algoritmo adaptivo para el registro de informacion desde nodos IoT para el monitoreo en la transportación dentro de la cadena de suministro. 
    Contesta siempre en español y agradece el usuario por preguntar. Si las preguntas son sobre otro tema, contesta que no puedes contestar.
    {context}
    Question: {question}
    Helpful Answer:"""
qa_prompt = ChatPromptTemplate.from_template(template)

In [14]:
#Do not modify
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)
conversation_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory,
    combine_docs_chain_kwargs={"prompt": qa_prompt}
)

C:\Users\L01356196\AppData\Local\Temp\ipykernel_27964\3110315447.py:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)


In [15]:
history = []

In [18]:
def handle_conversation(conversation,history):
    print('Escribe "exit" para salir')
    while True:
        question = input('Tu: ')
        if question.lower() == 'exit':
            break
        print(question)
        result = conversation({"question": question}, {"chat_history": history})
        history.append((question, result["answer"]))
        print("Bot: ",result['answer'])

handle_conversation(conversation_chain,history)

Escribe "exit" para salir
hola
Bot:  Muchas gracias por preguntar. Mi saludo en este contexto es simplemente una forma de iniciar la conversación de manera amigable y respetuosa. Mi objetivo es proporcionar ayuda y responder a tus preguntas de manera clara y concisa sobre el contenido del artículo relacionado con la implementación de un algoritmo adaptivo para el registro de información desde nodos IoT para el monitoreo en la transportación dentro de la cadena de suministro. ¿En qué puedo ayudarte específicamente sobre este tema?
cuáles son los sensores utilizados para el sistema IoT propuesto en el artículo?
Bot:  Muchas gracias por preguntar sobre el contenido del artículo. Desafortunadamente, el fragmento proporcionado no menciona específicamente los sensores utilizados en el sistema IoT propuesto. Sin embargo, en general, los sistemas IoT para el monitoreo en la cadena de suministro suelen utilizar una variedad de sensores, como sensores de temperatura, humedad, movimiento, presión

APIStatusError: Error code: 413 - {'error': {'message': 'Request too large for model `llama-3.1-70b-versatile` in organization `org_01jhr191svfy0as432pk4ybfds` service tier `on_demand` on tokens per minute (TPM): Limit 6000, Requested 6159, please reduce your message size and try again. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}